# 스태킹 연습

### 라이브러리 불러오기

In [41]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

import warnings

### 데이터 및 환경 세팅

In [18]:
warnings.filterwarnings('ignore')

cancer_data = load_breast_cancer()

X_feature = cancer_data.data
y_label = cancer_data.target

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_feature
                                                    , y_label
                                                    , test_size = 0.2
                                                    , random_state = 0
                                                    )

### 모델 학습

In [9]:
### 모델 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators = 100)

### 스태킹으로 학습된 데이터를 최종적으로 학습할 모델
lr_final = LogisticRegression()

In [ ]:
### 모델 학습
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [19]:
### 정확도 측정
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print(f'KNeighbors 정확도 {accuracy_score(y_test, knn_pred):.4f}')
print(f'RandomForest 정확도 {accuracy_score(y_test, rf_pred):.4f}')
print(f'의사결정트리 정확도 {accuracy_score(y_test, dt_pred):.4f}')
print(f'AdaBoost 정확도 {accuracy_score(y_test, ada_pred):.4f}')

KNeighbors 정확도 0.9211
RandomForest 정확도 0.9649
의사결정트리 정확도 0.9035
AdaBoost 정확도 0.9561


### 스태킹

In [22]:
### 예측값을 붙혀서 피처값을 새로 만듦
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

(4, 114)


In [38]:
### Transpose를 통해, 행/열 위치 교환, 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦
T_pred = np.transpose(pred)
lr_final.fit(T_pred, y_test)

final_pred = lr_final.predict(T_pred)

In [40]:
### 결과
print(f'스태킹 이후 최종 모델의 정확도 : {accuracy_score(y_test, final_pred):.4f}')

스태킹 이후 최종 모델의 정확도 : 0.9737


### CV세트를 기반의 스태킹

In [56]:
### 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    # 지정된 n_folds값으로 KFold 생성
    kf = KFold(n_splits = n_folds, shuffle = False)

    # 추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(f'{model.__class__.__name__} 모델 시작')

    for num, (i, j) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print(f'\t 폴드 세트, {num}, 시작')
        X_tr = X_train_n[i]
        y_tr = y_train_n[i]
        X_te = X_train_n[j]

        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행
        model.fit(X_tr, y_tr)

        # 폴드 세트 내뷍서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장
        train_fold_pred[j, :] = model.predict(X_te).reshape(-1,1)

        # 입력된 원본 테스트 데이터를 폴드 세트나 학습된 기반 모델에서 예측 후 데이터 저장
        test_pred[:, num] = model.predict(X_test_n)

    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1, 1)

    # train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred, test_pred_mean

In [57]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

KNeighborsClassifier 모델 시작
	 폴드 세트, 0, 시작
	 폴드 세트, 1, 시작
	 폴드 세트, 2, 시작
	 폴드 세트, 3, 시작
	 폴드 세트, 4, 시작
	 폴드 세트, 5, 시작
	 폴드 세트, 6, 시작
RandomForestClassifier 모델 시작
	 폴드 세트, 0, 시작
	 폴드 세트, 1, 시작
	 폴드 세트, 2, 시작
	 폴드 세트, 3, 시작
	 폴드 세트, 4, 시작
	 폴드 세트, 5, 시작
	 폴드 세트, 6, 시작
DecisionTreeClassifier 모델 시작
	 폴드 세트, 0, 시작
	 폴드 세트, 1, 시작
	 폴드 세트, 2, 시작
	 폴드 세트, 3, 시작
	 폴드 세트, 4, 시작
	 폴드 세트, 5, 시작
	 폴드 세트, 6, 시작
AdaBoostClassifier 모델 시작
	 폴드 세트, 0, 시작
	 폴드 세트, 1, 시작
	 폴드 세트, 2, 시작
	 폴드 세트, 3, 시작
	 폴드 세트, 4, 시작
	 폴드 세트, 5, 시작
	 폴드 세트, 6, 시작


In [58]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis = 1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis = 1)

print(f'''
▶ 원본 학습 피처 데이터 Shape
{X_train.shape}

▶ 원본 테스트 피처 데이터
{X_test.shape}

▶ 스태킹 학습 피처 데이터
{Stack_final_X_train.shape}

▶ 스태킹 테스트 피처 데이터
{Stack_final_X_test.shape}
''')


▶ 원본 학습 피처 데이터 Shape
(455, 30)

▶ 원본 테스트 피처 데이터
(114, 30)

▶ 스태킹 학습 피처 데이터
(455, 4)

▶ 스태킹 테스트 피처 데이터
(114, 4)



In [60]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final_pred = lr_final.predict(Stack_final_X_test)

print(f'''
▶ 최종 메타 모델의 예측 정확도
{accuracy_score(y_test, stack_final_pred):.4f}
''')


▶ 최종 메타 모델의 예측 정확도
0.9737

